In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from zipfile import ZipFile

zip_file = ZipFile('/content/drive/MyDrive/Master_projekat/Pets/Pets.zip')
zip_file.extractall()

In [4]:
pip install open-clip-torch joblib transformers annoy ipykernel streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 35.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_

In [5]:
import os
len(os.listdir('/content/Pets'))

37500

In [6]:
import sys

if '/content/drive/MyDrive/Master_projekat/Utils' in sys.path:
  sys.path.remove('/content/drive/MyDrive/Master_projekat/Utils')

In [7]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [8]:
import os
import pickle
import joblib
import torch
from tqdm import tqdm
import sys
from torch.utils.data import DataLoader

import sys
if '/content/drive/MyDrive/Master_projekat/Utils' not in sys.path:
  sys.path.append('/content/drive/MyDrive/Master_projekat/Utils')

from model_helpers import OPEN_CLIP_MODEL, get_open_clip_model, get_device
from general_helpers import check_file_exist
from pytorch_helpers import Images_with_Index

image_folder_path = os.path.join('/content', 'Pets')

device = get_device()
model, preprocess, tokenizer, model_name, pretrained, embedding_size = get_open_clip_model(OPEN_CLIP_MODEL.xlm_roberta_large_ViT_H.value)

cache_file_name = f"{model_name}-cache_using_pytorch.jlb"

if not check_file_exist(cache_file_name):
    print("Cache does not exist, making new cache file")
    cache = dict()
    joblib.dump(cache, cache_file_name)

cache = joblib.load(cache_file_name)
print("Cache loaded.")

cache_for_encoding = dict()

for photo_filename in os.listdir(image_folder_path):
    cnt = 0

    item = cache.get(photo_filename, None)
    if item is None:

        cache[photo_filename] = {
            "embedded_image": [],
            "image_path": os.path.join(image_folder_path, photo_filename),
        }
        cache_for_encoding[photo_filename] = {
            "embedded_image": [],
            "image_path": os.path.join(image_folder_path, photo_filename),
        }

if len(cache_for_encoding) != 0:

    with open('dict_of_images.pkl', 'wb') as file:
        pickle.dump(cache_for_encoding, file)

    with open('dict_of_images.pkl', 'rb') as file:
        dict_of_images = pickle.load(file)

    dataset = Images_with_Index(dict_of_images, preprocess)
    dataloader = DataLoader(dataset,
                            batch_size=512,
                            shuffle=True)

    print("Processing images...")
    for data in tqdm(dataloader):
        indexes = []
        visual_embeddings = []

        with torch.no_grad():
            X = data['image'].to(device)
            image_embedding = model.encode_image(X)
            index = data['index']
            indexes.extend(index)
            visual_embeddings.extend([torch.Tensor(x).cpu() for x in image_embedding.tolist()])

            del image_embedding

        image_embeddings = dict(zip(indexes, visual_embeddings))

        for index, visual_embeddings in image_embeddings.items():
            cache[index]["embedded_image"] = visual_embeddings

        joblib.dump(cache, cache_file_name)

There are 1 GPU(s) available.
GPU is: Tesla T4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

There are 1 GPU(s) available.
GPU is: Tesla T4
The file 'xlm-roberta-large-ViT-H-14-cache_using_pytorch.jlb' does not exist in the current folder.
Cache does not exist, making new cache file
Cache loaded.
Processing images...


100%|██████████| 74/74 [1:17:05<00:00, 62.50s/it]


# DATASET CHECK

In [ ]:
import os
import joblib
import sys

ROOT_DIR = os.path.split(os.environ['VIRTUAL_ENV'])[0]
sys.path.insert(0, ROOT_DIR)

from model_helpers import OPEN_CLIP_MODEL, get_open_clip_model, get_model_name

model_name = get_model_name(OPEN_CLIP_MODEL.xlm_roberta_large_ViT_H.value)
cache_file_name = os.path.join(ROOT_DIR, f"{model_name}-cache_using_pytorch.jlb")

cache = joblib.load(cache_file_name)

print("Len of products in cache", len(cache.keys()))